Imports

In [ ]:
!pip3 install pyconll
import pyconll # https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

Carregando o spaCy

In [ ]:
!pip3 install --upgrade spacy
import spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download pt_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 568.2 MB 7.4 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


In [ ]:
nlp = spacy.load('pt_core_news_lg') # https://spacy.io/models/pt#pt_core_news_lg

In [ ]:
# # Process whole documents
# text = ("""My name is Shaurya Uppal. 
# I enjoy writing articles on GeeksforGeeks checkout
# my other article by going to my profile section.""")
  
# doc = nlp(text)
  
# # Token and Tag
# for token in doc:
#   print(token, token.pos_)
  
# # You want list of Verb tokens
# print("Verbs:", [token.text for token in doc if token.pos_ == "VERB"])

Salvando o modelo pt_core_news_lg no Google Drive e mudando o diretório de trabalho. 

Obs.: Quando eu mudava o diretório de trabalho antes de fazer o download do modelo de linguagem português no spaCy, depois dava problema na hora de carregar usando a função *spacy.load('pt_core_news_lg')* porque o Colaboratory se perdia. Então, precisei mudar o diretório depois de carregar o modelo para poder apontar para o dataset.

In [ ]:
from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd "drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning/"

/content/drive/MyDrive/Doutorado/Disciplinas/[2022.1] [UFF] Processamento de Linguagem Natural - Professora: Aline Marins Paes Carvalho/Trabalhos/Trabalho 2 - POS  e Transfer Learning


In [ ]:
# nlp.to_disk('./spacy__pt_core_news_lg/') # Comentei porque já salvei.

Carregando o dataset em português

In [ ]:
dataset = 'train' # 'train' | 'dev' | 'test'

In [ ]:
set_X = list()
set_y = list()
data = pyconll.load_from_file( f'./datasets/pt_bosque-ud-{dataset}.conllu' )
t=0
for sentence in data:
    X_sentence = list()
    y_sentence = list()
    for token in sentence:
        if token.upos and token.form:
            X_sentence.append( token.form )
            y_sentence.append( token.upos )
    set_X.append( ' '.join(str(t) for t in X_sentence) )
    set_y.append( [str(tag) for tag in y_sentence] )

In [ ]:
set_X[0]

'PT em o governo'

In [ ]:
set_y[0]

['PROPN', 'ADP', 'DET', 'NOUN']

In [ ]:
len(set_y)

7018

In [ ]:
sentence_index = 0
sentence = set_X[ sentence_index ]

doc = nlp( sentence )  
print( f'TOKEN{"":{11}}TAG_UD{"":{10}}TAG_SPACY' )
print()
for token in doc:
  tag_ud = set_y[ sentence_index ][ token.i ]
  print( f'{token.text:{15}} {tag_ud:{15}} {token.pos_}' )

TOKEN           TAG_UD          TAG_SPACY

PT              PROPN           PROPN
em              ADP             ADP
o               DET             DET
governo         NOUN            NOUN


In [ ]:
%%time

set_y_spacy = []
for sentence in set_X:

  tag_list = []
  doc = nlp( sentence )  
  for token in doc:
    tag_list.append( token.pos_ )

  set_y_spacy.append( tag_list )
print(set_y_spacy)

[['PROPN', 'ADP', 'DET', 'NOUN'], ['PROPN', 'NOUN', 'PROPN', 'VERB', 'ADV', 'VERB', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'VERB', 'DET', 'NOUN', 'ADJ', 'PUNCT', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'NUM', 'SYM', 'PUNCT', 'ADP', 'DET', 'NOUN', 'VERB', 'DET', 'PROPN', 'VERB', 'ADP', 'DET', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT'], ['VERB', 'NOUN', 'PUNCT', 'ADV', 'PUNCT', 'DET', 'NOUN', 'PUNCT'], ['ADV', 'ADV', 'ADP', 'PRON', 'PRON', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'PROPN', 'AUX', 'PUNCT', 'ADV', 'PUNCT', 'VERB', 'PUNCT'], ['CCONJ', 'PROPN', 'CCONJ', 'DET', 'NOUN', 'ADV', 'VERB', 'DET', 'NOUN', 'SCONJ', 'PRON', 'VERB', 'PUNCT'], ['PRON', 'PRON', 'VERB', 'NOUN', 'PUNCT', 'CCONJ', 'ADV', 'ADV', 'VERB', 'PRON', 'PRON', 'AUX', 'VERB', 'PUNCT'], ['PRON', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'PROPN', 'PUNCT'], ['PUNCT', 'PROPN', 'PUNCT', 'VERB', 'ADP', 'PROPN'], ['ADP', 'DET', 'ADJ', 'NOUN', 'NUM

In [ ]:
len(set_y)

7018

In [ ]:
len(set_y_spacy)

7018

In [ ]:
tags_real = []
tags_pred = []
for i, tag_list in enumerate(set_y):
  # print(f'{i} - {tag_list}')
  for j, tag in enumerate(tag_list):
    # print(f'{j} - {tag}')
    tags_real.append( tag )
    tags_pred.append( set_y_spacy[i][j] )

print( len(tags_real) )
print( len(tags_pred) )

171776
171776


In [ ]:
# y_real = np.array( tags_real )
# y_pred = np.array( tags_pred )
y_real = tags_real
y_pred = tags_pred

In [ ]:
print( classification_report( y_real, y_pred ) )
f1 = f1_score( y_real, y_pred, average='weighted' )
acc = accuracy_score( y_real, y_pred )
print( f'F1: {f1:.2}' )
print( f'Accuracy: {acc:.2}' )

              precision    recall  f1-score   support

         ADJ       0.98      0.97      0.97      8638
         ADP       0.98      0.98      0.98     25482
         ADV       0.96      0.97      0.97      6345
         AUX       0.97      0.98      0.98      3713
       CCONJ       0.98      0.98      0.98      4029
         DET       0.98      0.99      0.98     26307
        INTJ       0.68      0.56      0.61        27
        NOUN       0.98      0.97      0.98     31136
         NUM       0.97      0.97      0.97      3580
        PART       1.00      0.33      0.50         3
        PRON       0.96      0.88      0.92      5525
       PROPN       0.94      0.98      0.96     14249
       PUNCT       0.99      0.99      0.99     22586
       SCONJ       0.85      0.92      0.89      4056
         SYM       1.00      1.00      1.00       321
        VERB       0.99      0.98      0.98     15678
           X       0.91      0.90      0.91       101

    accuracy              